### Import modules and merge the labs, demographics, and DX csv files

In [ ]:
# Import required packages
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import sklearn
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, roc_auc_score, recall_score, precision_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import RepeatedKFold, ShuffleSplit, KFold, RepeatedStratifiedKFold, cross_val_predict
import statsmodels.api as sm
from statsmodels.stats.proportion import proportion_confint
import scipy.stats

df = pd.read_csv("one_encounter_df.csv")

In [ ]:
# Run to drop Barthel index
df.drop("Barthel", inplace = True, axis = 1)

In [ ]:
# Seperate dataset as response variable and feature variables
X = df.loc[:, ~df.columns.str.contains('DX')]
y = df['DX']

In [ ]:
def evaluate_threshold(threshold):
    print('Sensitivity:', tpr[thresholds > threshold][-1])
    print('Specificity:', 1 - fpr[thresholds > threshold][-1])
    
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return (m-h, m+h)


# Set up inner and outer CV loops for nested CV
cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
cv_outer = RepeatedStratifiedKFold(n_splits=10, random_state=5)

# Logistic Regression
model = LogisticRegression()
param_grid = {}
logistic = GridSearchCV(model, param_grid, scoring='roc_auc', n_jobs=1, cv=cv_inner, refit=True)

# Random Forest
model = RandomForestClassifier(max_features = 'sqrt', min_samples_split = 20, n_jobs = 32,
                               min_samples_leaf = 2, criterion = 'entropy')
param_grid = {'n_estimators': [200, 300], 'max_depth' : [10,11,12],'max_samples' : [.25,.5,.75]}
rfc = GridSearchCV(model, param_grid, scoring='roc_auc', n_jobs=1, cv=cv_inner, refit=True)

# Gradient boosted model 
model = GradientBoostingClassifier(min_samples_split=20, min_samples_leaf=2, subsample=.5, max_features='sqrt',random_state=10)
param_grid = {"n_estimators" : [300,400], "learning_rate" : [.01, 0.1],"max_depth":[7,8,9]}
gbm = GridSearchCV(model, param_grid, scoring='roc_auc', n_jobs=1, cv=cv_inner, refit=True)
    
# KNN
model = KNeighborsClassifier(n_neighbors=200)
param_grid = {'n_neighbors' : [200,300,400]}
KNN = GridSearchCV(model, param_grid, scoring='roc_auc', n_jobs=1, cv=cv_inner, refit=True)
    
# Naive Bayes
model = GaussianNB()
param_grid = {}
NB = GridSearchCV(model, param_grid, scoring='roc_auc', n_jobs=1, cv=cv_inner, refit=True)

# Adaboost
model = Ada=AdaBoostClassifier()
param_grid = {'n_estimators':[300,400],'learning_rate':[.05, .1]}
Ada = GridSearchCV(model, param_grid, scoring='roc_auc', n_jobs=1, cv=cv_inner, refit=True)

# Decision Tree
model = sklearn.tree.DecisionTreeClassifier(criterion = 'entropy')
param_grid = {'max_depth' : [8,9,10], 'min_samples_leaf' : [5, 10], 'min_samples_split' : [20, 35, 50]}
DT = GridSearchCV(model, param_grid, scoring='roc_auc', n_jobs=1, cv=cv_inner, refit=True)

from sklearn import svm
model = svm.SVC(probability = True, kernel = 'rbf')
param_grid = {'gamma' : [.1,1], 'C': [.1,1,10]}
SVM = GridSearchCV(model, param_grid, scoring='roc_auc', n_jobs=1, cv=cv_inner, refit=True)

In [ ]:
# Set up table for specified sensitivity
sensitivity_result_table = pd.DataFrame(columns=['classifiers', 'accuracy', 'fpr','tpr','auc', 'sensitivity', 'specificity','ROC_plot'])
# Set up table for specified specificity
specificity_result_table = pd.DataFrame(columns=['classifiers','accuracy', 'fpr','tpr','auc', 'sensitivity', 'specificity','ROC_plot'])

classifiers = [rfc, gbm,logistic, KNN, DT,NB,SVM]

# Train the models and record the results
for cls in classifiers:
    
    # Calculate AUC 
    auc = cross_val_score(cls, X, y, scoring='roc_auc', cv=cv_outer, n_jobs=-1)
    auc_mean = np.mean(auc)
    auc_CI = mean_confidence_interval(auc, confidence=0.95)
    auc_CI = tuple([float("{0:.3f}".format(n)) for n in auc_CI])
    
    # Get true positive rate and false positive rate for sensitivity / specificity thresholds
    yproba = cross_val_predict(cls, X, y, cv=10, method='predict_proba')[:,1]
    fpr, tpr, _ = roc_curve(y,  yproba)
    pred_labels = np.where(yproba > 0.5, 1, 0)
    model_name = cls.estimator.__class__.__name__
    print(f'Confusion Matrix and Classification Report for {model_name}:')
    print(confusion_matrix(y, pred_labels))
    print(classification_report(y, pred_labels))
    print()
    
    # Get accuracy score using the predictions and true labels
    accuracy = round(metrics.accuracy_score(y, pred_labels),2)

    
    # Define sensitivity and specificty thresholds to evaluate
    select_sensitivity = .9
    select_specificity = .5
    x_fpr = fpr
    y_tpr = tpr
    
    # Determine specificities at a set sensitivity value
    specificity = 1 - np.interp(select_sensitivity, y_tpr, x_fpr) 
    # Determine sensitivities at a set sensitivity value
    sensitivity = np.interp(1 - select_specificity, x_fpr, y_tpr)
    
    # Calculate CI intervals for sensitivity theshold adjustment 
    specificity_CI = proportion_confint(count= len(df)*specificity, nobs = len(df), alpha=.05)
    specificity_CI = tuple([float("{0:.3f}".format(n)) for n in specificity_CI])
    sensitivity_CI = proportion_confint(count = len(df)*select_sensitivity, nobs = len(df), alpha=.05)
    sensitivity_CI = tuple([float("{0:.3f}".format(n)) for n in sensitivity_CI])
    
    sensitivity_result_table = sensitivity_result_table.append({'classifiers':cls.estimator.__class__.__name__,
                                        'fpr':list(fpr), 
                                        'tpr':list(tpr), 
                                        'auc':f'{round(auc_mean,3)} {auc_CI}',
                                        'ROC_plot': round(auc_mean,3),
                                        'sensitivity': f'{select_sensitivity} {sensitivity_CI}',
                                        'specificity' : f'{round(specificity,2)} {specificity_CI}',
                                        'accuracy' : accuracy}, ignore_index=True)
                

    
    # Calculate CI intervals for specificity threshold adjustment 
    specificity_CI = proportion_confint(count= len(df)*select_specificity, nobs = len(df), alpha=.05)
    specificity_CI = tuple([float("{0:.3f}".format(n)) for n in specificity_CI])
    sensitivity_CI = proportion_confint(count = len(df)*sensitivity, nobs = len(df), alpha=.05)
    sensitivity_CI = tuple([float("{0:.3f}".format(n)) for n in sensitivity_CI])
    
    specificity_result_table = specificity_result_table.append({'classifiers':cls.estimator.__class__.__name__,
                                        'fpr':list(fpr), 
                                        'tpr':list(tpr), 
                                        'auc':f'{round(auc_mean,3)} {auc_CI}',
                                        'ROC_plot': round(auc_mean,3),
                                        'sensitivity': f'{round(sensitivity,2)} {sensitivity_CI}',
                                        'specificity' : f'{select_specificity} {specificity_CI}',
                                        'accuracy' : accuracy}, ignore_index=True)

specificity_result_table.set_index('classifiers', inplace=True)
sensitivity_result_table.set_index('classifiers', inplace=True)

In [ ]:
sensitivity_result_table
specificity_result_table

In [ ]:
import ast

specificity_result_table.to_csv('specificity_one_encounter')
specificity_result_table = pd.read_csv("specificity_one_encounter.csv", index_col = 0)

# Plot ROC curve and save image
fig = plt.figure(figsize=(12,10))

for i in specificity_result_table.index[:7]:
    a = ast.literal_eval(specificity_result_table.loc[i]['fpr'])
    b = ast.literal_eval(specificity_result_table.loc[i]['tpr'])
    plt.plot(a, b, linewidth=3.0,
             label="{}, AUC={:.3f}".format(i, specificity_result_table.loc[i]['ROC_plot']))
    
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1), fontsize = 15)
plt.xlabel("Flase Positive Rate", fontsize=20, labelpad=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1), fontsize = 15)
plt.ylabel("True Positive Rate", fontsize=20, labelpad=15)

plt.title('ROC Curve Analysis', fontsize=20, pad = 15, fontweight='bold')
plt.legend(prop={'size':15}, loc='lower right')



plt.show()
# plt.savefig('ROC_one_encounter_no_barthel.png')

### Logistic Final Model

In [ ]:
# Get model Coefficients and Odds Ratios from Logistic Regression
logistic = LogisticRegression(random_state = 1)
logistic.fit(X, y)


res = sm.Logit(y, sm.add_constant(X)).fit()
params = res.params
conf = res.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['2.5%', '97.5%', 'Odds Ratio']
conf = round(np.exp(conf),2)
conf['95% CI'] = [f'{j[0]}-{j[1]}' for j in conf[['2.5%', '97.5%']].values] 
conf.to_csv('Odds_Ratio.csv')
conf

In [ ]:
res.summary()

### Random Forest Final Model 

In [ ]:
param_grid = { 
    'n_estimators': [200, 300],
    'max_depth' : [10,11,12],
    'min_samples_leaf' : [2,10],
    'min_samples_split' : [10,25],
    'max_samples' : [.25,.5]
}

rfc = GridSearchCV(RandomForestClassifier(min_samples_split = 20, n_jobs = 32),
                                                   param_grid, scoring = 'roc_auc', cv = 10)
rfc.fit(X, y)
rfc.best_estimator_

In [ ]:
# Run with Barthel
rfc = RandomForestClassifier(n_estimators=200, max_features = 'sqrt', max_depth = 10,
                            min_samples_split = 10, n_jobs = 32, min_samples_leaf = 2,
                            criterion = 'entropy', max_samples = .25, random_state=10)

rfc.fit(X, y)
feature_importances = pd.DataFrame(rfc.feature_importances_, index = X.columns,
                      columns=['importance']).sort_values('importance', ascending=False) 

Barthel = feature_importances.loc['Barthel', 'importance']
feature_importances['Scaled Relative Importance'] = feature_importances['importance'].apply(lambda x : round((x/Barthel)*100,1))
feature_importances

In [ ]:
# Run without Barthel
rfc = RandomForestClassifier(n_estimators=200, max_features = 'sqrt', max_depth = 10,
                            min_samples_split = 10, n_jobs = 32, min_samples_leaf = 2,
                            criterion = 'entropy', max_samples = .25, random_state=10)

rfc.fit(X, y)
feature_importances = pd.DataFrame(rfc.feature_importances_, index = X.columns,
                      columns=['importance']).sort_values('importance', ascending=False) 

Dementia = feature_importances.loc['Dementia', 'importance']
feature_importances['Scaled Relative Importance'] = feature_importances['importance'].apply(lambda x : round((x/Dementia)*100,1))
feature_importances

### GBM Final Model

In [ ]:
# Grid search on entire dataset to get optimal parameters for the final model
# Train final model on all data
# Get relative importances

parameters = {
    "n_estimators" : [300,400],
    "learning_rate" : [.01, 0.1],
    "max_depth":[7,8,9],
}

gbm = GridSearchCV(GradientBoostingClassifier(n_estimators = 100), parameters,
                   cv=10, scoring = 'roc_auc', n_jobs = 32)

gbm.fit(X, y)
gbm.best_params_

In [ ]:
# Run with Barthel
gbm = GradientBoostingClassifier(n_estimators=400, learning_rate=.01,
     max_depth=7, min_samples_split=20, min_samples_leaf=2, subsample=.25, max_features='sqrt',random_state=10)


gbm.fit(X, y)
feature_importances = pd.DataFrame(gbm.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance', ascending=False) 

Barthel = feature_importances.loc['Barthel', 'importance']
feature_importances['Scaled Relative Importance'] = feature_importances['importance'].apply(lambda x : round((x/Barthel)*100,1))
feature_importances

In [ ]:
# Run without Barthel
gbm = GradientBoostingClassifier(n_estimators=400, learning_rate=.01,
     max_depth=7, min_samples_split=20, min_samples_leaf=2, subsample=.25, max_features='sqrt',random_state=10)


gbm.fit(X, y)
feature_importances = pd.DataFrame(gbm.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance', ascending=False) 

Dementia = feature_importances.loc['Dementia', 'importance']
feature_importances['Scaled Relative Importance'] = feature_importances['importance'].apply(lambda x : round((x/Dementia)*100,1))
feature_importances

### KNN Final Model

In [ ]:
model = KNeighborsClassifier()
param_grid = {'n_neighbors' : [200,300,400]}
KNN = GridSearchCV(model, param_grid, scoring='roc_auc', n_jobs=1, cv=10, refit=True)
KNN.fit(X,y)
KNN.best_estimator_

In [ ]:
KNN = KNeighborsClassifier(n_neighbors=300)
KNN.fit(X,y)

### Naive Bayes Final Model

In [ ]:
NB = GaussianNB()
NB.fit(X,y)

### DT Final Model

In [ ]:
param_grid = { 
    'max_depth' : [8,9,10],
    'min_samples_leaf' : [2, 10], 
    'min_samples_split' : [20, 35, 50],
    'criterion' : ['gini', 'entropy']
}

DT = GridSearchCV(sklearn.tree.DecisionTreeClassifier(), 
                  param_grid, scoring = 'roc_auc', cv = 10)
DT.fit(X, y)
DT.best_estimator_

In [ ]:
DT = sklearn.tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 8,
                                        min_samples_split = 50, min_samples_leaf = 2, random_state = 10)
DT.fit(X,y)

### SVC Final Model

In [ ]:
from sklearn import svm
model = svm.SVC(probability = True, kernel = 'rbf')
param_grid = {'gamma' : [.1,1], 'C': [.1,1,10]}

SVM = GridSearchCV(model, 
                  param_grid, scoring = 'roc_auc', cv = 10)
SVM.fit(X, y)
SVM.best_estimator_

In [ ]:
svm = svm.SVC(gamma=.1, C=1, probability = True, kernel = 'rbf')
svm.fit(X,y)

### Predictied Probabilities

In [ ]:
test_cases = pd.read_csv('test_cases.csv', index_col = 0)
test_cases = pd.read_csv('test_cases_no_barthel.csv', index_col = 0)

pred_log = logistic.predict_proba(test_cases)[:,1]
pred_rfc = rfc.predict_proba(test_cases)[:,1]
pred_gbm = gbm.predict_proba(test_cases)[:,1]

In [ ]:
predicted_probabilities = pd.DataFrame(data = {'RF':pred_rfc,
                                               'GBM':pred_gbm,
                                               'LR':pred_log})
predicted_probabilities.index = ['Case 1', 'Case 2', 'Case 3'] 
predicted_probabilities.round(2)